# Parte 3 (Computational Complexity Analysis)
**Objetivo:** medir tiempos reales, comparar brute force vs optimizado, analizar complejidad y proponer optimizaciones.

## Importaciones

In [1]:
import os, random, time
import numpy as np
from PIL import Image


## Reutilización de funciones (Generador + Detector)

Para poder medir tiempos en distintas configuraciones (N, W, H), necesitamos:
- generar datasets (ruido + 1 imagen con mensaje)
- ejecutar detector brute force y optimizado

Nota: el objetivo aquí NO es “adivinar” la imagen, sino medir tiempos.


In [2]:
END_DELIMITER = "1111111111111110"
HEADER = "FLAG{"

def create_noise_image(filename, width, height):
    arr = np.random.randint(0, 256, size=(height, width, 3), dtype=np.uint8)
    Image.fromarray(arr, mode="RGB").save(filename, format="PNG")

def text_to_binary(message: str) -> str:
    return "".join(format(ord(ch), "08b") for ch in message)

def hide_lsb(image_path: str, message: str):
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)
    flat = data.reshape(-1)

    bits = text_to_binary(message) + END_DELIMITER
    if len(bits) > len(flat):
        raise ValueError("Mensaje demasiado largo para esta imagen.")

    for i, b in enumerate(bits):
        flat[i] = (flat[i] & 0xFE) | int(b)

    Image.fromarray(flat.reshape(data.shape), mode="RGB").save(image_path, format="PNG")

def generate_dataset(folder: str, N: int, W: int, H: int, flag: str):
    os.makedirs(folder, exist_ok=True)

    # Limpieza opcional: borrar PNG anteriores (para que N sea exacto)
    for f in os.listdir(folder):
        if f.lower().endswith(".png"):
            os.remove(os.path.join(folder, f))

    files = []
    for i in range(N):
        fname = os.path.join(folder, f"img_{i:03d}.png")
        create_noise_image(fname, W, H)
        files.append(fname)

    secret = random.choice(files)
    hide_lsb(secret, flag)


In [3]:
def extract_lsb(image_path: str, num_bits: int | None = None) -> str:
    img = Image.open(image_path).convert("RGB")
    data = np.array(img, dtype=np.uint8)
    flat = data.reshape(-1)

    if num_bits is None:
        relevant = flat
    else:
        relevant = flat[:num_bits]

    bits = (relevant & 1).astype(np.uint8)
    return "".join(bits.astype(str))

def binary_to_text(bits: str, end_delimiter: str = END_DELIMITER) -> str:
    end_idx = bits.find(end_delimiter)
    if end_idx != -1:
        bits = bits[:end_idx]

    out = []
    for i in range(0, len(bits) - 7, 8):
        out.append(chr(int(bits[i:i+8], 2)))
    return "".join(out)

def search_brute_force(folder: str, header: str = HEADER):
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".png")])
    for fname in files:
        path = os.path.join(folder, fname)
        bits = extract_lsb(path, None)
        text = binary_to_text(bits)
        if header in text:
            return fname, text
    return None, None

def search_optimized(folder: str, header: str = HEADER):
    files = sorted([f for f in os.listdir(folder) if f.lower().endswith(".png")])
    header_bits = len(header) * 8  # 40 bits

    for fname in files:
        path = os.path.join(folder, fname)

        bits_prefix = extract_lsb(path, header_bits)
        text_prefix = binary_to_text(bits_prefix, end_delimiter="")  # sin delimiter para prefix

        if text_prefix.startswith(header):
            bits_all = extract_lsb(path, None)
            text_full = binary_to_text(bits_all, end_delimiter=END_DELIMITER)
            return fname, text_full

    return None, None


## Experimento: medir tiempos (tabla del enunciado)

Mediremos para:
- N=10, 50, 100 con 200×200
- N=100 con 500×500
- N=100 con 1000×1000

Recomendación: hacer varias repeticiones y tomar mínimo o media.
Aquí haremos `reps=3` y tomaremos el mínimo (reduce ruido por procesos en background).


In [4]:
def time_once(fn, *args, **kwargs):
    t0 = time.perf_counter()
    result = fn(*args, **kwargs)
    t1 = time.perf_counter()
    return (t1 - t0), result

def bench_detector(folder, reps=3):
    # Brute force
    brute_times = []
    brute_res = None
    for _ in range(reps):
        dt, res = time_once(search_brute_force, folder)
        brute_times.append(dt)
        brute_res = res
    brute_best = min(brute_times)

    # Optimizado
    opt_times = []
    opt_res = None
    for _ in range(reps):
        dt, res = time_once(search_optimized, folder)
        opt_times.append(dt)
        opt_res = res
    opt_best = min(opt_times)

    return brute_best, opt_best, brute_res, opt_res
FLAG = "FLAG{LUCIACANTOSBURGOS}"
BASE = "bench_datasets"

configs = [
    (10, 200, 200),
    (50, 200, 200),
    (100, 200, 200),
    (100, 500, 500),
    (100, 1000, 1000),
]

results = []

for (N, W, H) in configs:
    folder = os.path.join(BASE, f"N{N}_W{W}_H{H}")
    print(f"\n== Generando dataset: N={N}, {W}x{H} ==")
    generate_dataset(folder, N, W, H, FLAG)

    print("   Midiendo brute vs optimizado...")
    brute_t, opt_t, brute_res, opt_res = bench_detector(folder, reps=3)

    results.append({
        "N": N,
        "WxH": f"{W}x{H}",
        "BruteForce_sec": brute_t,
        "Optimized_sec": opt_t,
        "Speedup": (brute_t / opt_t) if opt_t > 0 else None
    })

results



== Generando dataset: N=10, 200x200 ==
   Midiendo brute vs optimizado...

== Generando dataset: N=50, 200x200 ==
   Midiendo brute vs optimizado...

== Generando dataset: N=100, 200x200 ==
   Midiendo brute vs optimizado...

== Generando dataset: N=100, 500x500 ==
   Midiendo brute vs optimizado...

== Generando dataset: N=100, 1000x1000 ==
   Midiendo brute vs optimizado...


[{'N': 10,
  'WxH': '200x200',
  'BruteForce_sec': 0.10770709998905659,
  'Optimized_sec': 0.0037595999892801046,
  'Speedup': 28.64855311633314},
 {'N': 50,
  'WxH': '200x200',
  'BruteForce_sec': 1.054441599873826,
  'Optimized_sec': 0.020948999794200063,
  'Speedup': 50.3337443425704},
 {'N': 100,
  'WxH': '200x200',
  'BruteForce_sec': 0.6986632000189275,
  'Optimized_sec': 0.03920109989121556,
  'Speedup': 17.822540743952146},
 {'N': 100,
  'WxH': '500x500',
  'BruteForce_sec': 12.282406099839136,
  'Optimized_sec': 0.4382419001776725,
  'Speedup': 28.02654446062686},
 {'N': 100,
  'WxH': '1000x1000',
  'BruteForce_sec': 13.95277399988845,
  'Optimized_sec': 1.7921787998639047,
  'Speedup': 7.785369406751158}]

In [5]:
for r in results:
    print(f"{r['N']:>4} | {r['WxH']:<9} | brute={r['BruteForce_sec']:.4f}s | opt={r['Optimized_sec']:.4f}s | x{r['Speedup']:.1f}")


  10 | 200x200   | brute=0.1077s | opt=0.0038s | x28.6
  50 | 200x200   | brute=1.0544s | opt=0.0209s | x50.3
 100 | 200x200   | brute=0.6987s | opt=0.0392s | x17.8
 100 | 500x500   | brute=12.2824s | opt=0.4382s | x28.0
 100 | 1000x1000 | brute=13.9528s | opt=1.7922s | x7.8


## Timing Results

| N (images) | W x H | Brute Force (sec) | Optimized (sec) | Speedup |
|------------|-------|-------------------|-----------------|---------|
| 10  | 200x200   | 0.1077 | 0.0038 | ×28.6 |
| 50  | 200x200   | 1.0544 | 0.0209 | ×50.3 |
| 100 | 200x200   | 0.6987 | 0.0392 | ×17.8 |
| 100 | 500x500   | 12.2824 | 0.4382 | ×28.0 |
| 100 | 1000x1000 | 13.9528 | 1.7922 | ×7.8 |


## 4.2.1 Question 1: Theoretical Complexity

Sea:
- N = nº de imágenes
- W, H = dimensiones
- k = tamaño del header en bytes (p.ej. k=5 para `"FLAG{"`)

### (a) Complejidad brute force
Brute force extrae LSB de TODOS los píxeles (y 3 canales) para TODAS las imágenes:

\[
O(N \cdot W \cdot H)
\]

(más precisamente, \(O(N \cdot W \cdot H \cdot 3)\), pero el 3 es constante).

### (b) Complejidad optimizada (early termination)
Solo extrae lo necesario para comprobar el header (k bytes = 8k bits) por imagen:

\[
O(N \cdot k)
\]

y cuando encuentra la correcta, paga un coste extra de extracción completa una vez:
\[
O(N \cdot k + W\cdot H)
\]
(ese segundo término ocurre solo para 1 imagen).

### (c) Si W=H=1000 y k=5, ¿cuántas veces más rápido?
En brute force, bits potenciales por imagen ≈ \(W\cdot H\cdot 3\).
En optimizado, bits leídos ≈ \(8k = 40\).

Factor teórico:
\[
\frac{W\cdot H\cdot 3}{8k}
= \frac{1000\cdot1000\cdot 3}{40}
= \frac{3,000,000}{40}
= 75,000
\]

→ Teóricamente, el optimizado puede ser ~**75.000×** más rápido en trabajo de extracción de bits.
(En la práctica, el speedup real puede ser menor por coste de lectura/decodificación PNG.)


## 4.2.2 Question 2: Bottlenecks

### (a) ¿Qué tarda más: I/O (leer) o CPU (calcular bits)?
Depende del tamaño y del formato. En PNG (comprimido), **abrir/decodificar** la imagen suele tener un coste importante (I/O + descompresión), y a tamaños grandes puede dominar.

En nuestro código, aunque el método “optimizado” procesa menos bits, **PIL normalmente decodifica la imagen completa al convertir a array**, así que parte del coste no desaparece.

### (b) ¿Cómo verificarlo empíricamente?
Hacemos micro-benchmarks separando:
1) Solo abrir + convertir a array (I/O + decode)
2) Solo extraer LSB desde un array ya en memoria (CPU)

Si (1) >> (2), el cuello es I/O/decode.
Si (2) >> (1), el cuello es CPU.

### (c) Si el cuello es I/O, ¿ayuda un procesador más rápido?
No demasiado. Mejoraría poco si el límite real es disco/decodificación.
En ese caso ayudan más:
- SSD más rápido
- reducir tamaño de datos
- paralelizar lectura
- o usar formatos/estrategias que permitan “early decode” real (no siempre posible con PNG).


In [6]:
def bench_io_cpu_one_image(image_path, reps=10):
    # 1) I/O + decode (open + np.array)
    io_times = []
    for _ in range(reps):
        t0 = time.perf_counter()
        img = Image.open(image_path).convert("RGB")
        arr = np.array(img, dtype=np.uint8)
        t1 = time.perf_counter()
        io_times.append(t1 - t0)

    # 2) CPU (extraer LSB) con array ya en memoria
    arr = arr  # usamos el último
    flat = arr.reshape(-1)
    cpu_times = []
    for _ in range(reps):
        t0 = time.perf_counter()
        _ = (flat & 1)
        t1 = time.perf_counter()
        cpu_times.append(t1 - t0)

    return min(io_times), min(cpu_times)

# Prueba sobre una imagen de un dataset grande (si existe)
test_folder = os.path.join(BASE, "N100_W1000_H1000")
test_img = os.path.join(test_folder, sorted([f for f in os.listdir(test_folder) if f.endswith(".png")])[0])
io_t, cpu_t = bench_io_cpu_one_image(test_img, reps=10)
io_t, cpu_t, io_t/cpu_t


(0.018867400009185076, 0.001223200000822544, 15.42462393435059)

## 4.2.3 Question 3: Scalability

### (a) Si tuviera 1 millón de imágenes, ¿cuánto tardaría el optimizado?
Usamos el tiempo medio por imagen del optimizado (medido).
Si para N=100 el optimizado tardó `T_100`, entonces aprox:

\[
t_{img} \approx \frac{T_{100}}{100}
\quad\Rightarrow\quad
T_{1,000,000} \approx 1,000,000 \cdot t_{img}
\]

(Esto asume comportamiento lineal con N, que es lo esperable.)

### (b) ¿Cómo usar multiprocessing?
Dividir la lista de imágenes en P bloques y que cada proceso analice un bloque.
Cuando uno encuentra el mensaje, se puede cancelar el resto (en la práctica, con colas/eventos).

### (c) Complejidad con P procesadores
Idealmente:
\[
O\left(\frac{N\cdot k}{P}\right)
\]
pero en la práctica hay overhead (arranque de procesos, lectura de disco, contención I/O).


In [7]:
# Tomamos como referencia la config N=100 y el tiempo optimizado medido
ref = next(r for r in results if r["N"] == 100 and r["WxH"] == "200x200")
T100 = ref["Optimized_sec"]
t_img = T100 / 100
T1M = t_img * 1_000_000

T100, t_img, T1M


(0.03920109989121556, 0.0003920109989121556, 392.0109989121556)

## 4.2.4 Question 4: Steganography Security

### (a) Si el atacante NO usa un header predecible como "FLAG{", ¿se puede automatizar la búsqueda?
Se vuelve muchísimo más difícil.
Con header, la detección es un problema de “pattern matching” rápido.
Sin header, no puedes distinguir fácilmente entre:
- bits aleatorios del LSB (ruido natural o aleatorio)
- bits de un mensaje cifrado (que también parece aleatorio)

La búsqueda automática se puede hacer, pero ya no es por firma ("FLAG{"), sino por **detección estadística** (esteganoanálisis), y tendrá falsos positivos/negativos.

### (b) ¿Cómo distinguir “random noise” vs “encrypted message”?
Un mensaje cifrado bien hecho produce bits ~uniformes (50/50), igual que el ruido.
Así que por distribución simple de 0/1 puede ser indistinguible.

Estrategias:
- buscar estructura (delimitadores, longitudes, redundancia) → si existen
- tests estadísticos más finos (chi-cuadrado, RS steganalysis, correlaciones locales)
- comparar con el modelo esperado de una imagen “natural” (no ruido puro)

En este lab, como las imágenes son *ruido aleatorio*, la detección estadística es todavía más difícil; por eso el header es clave.

### (c) ¿Qué puede hacer el atacante para dificultar detección?
- Insertar bits en posiciones pseudoaleatorias con una clave (no secuencial)
- Cifrar + comprimir el mensaje antes de insertar (menos patrones)
- Usar LSB matching (no solo setear el bit, sino ajustar ±1 aleatoriamente)
- Repartir el payload en menos píxeles o en canales específicos
- Cambiar de dominio (p.ej. JPEG/DCT) en lugar de LSB directo en PNG
- Usar técnicas adaptativas (inserta más en zonas con textura donde se nota menos)
